In [1]:
# TODO account for main stat of 5-star
# nbd cuz false positives (of good runes) are unlikely:
# they're dragged down by low subs anyways

In [2]:
import numpy as np
import pandas as pd

In [3]:
weights = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
weights_off = {'sub_acc': 1, 'sub_res': 0.5, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 0.5, 'sub_deff': 0.25, 'sub_hpp': 0.5, 'sub_hpf': 0.25, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
weights_def = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}

In [4]:
df = pd.read_csv('runes-data.csv', sep=';')

In [5]:
# Clean entries
df = df.replace('-', np.nan)

In [6]:
# For rune counting, drop Tricaru Icaru runes
df = df[~(df['monster_n'].str.contains('Icaru'))]

In [7]:
# Count rune sets at a glance
# Good for adjusting cutoffs
df_not_inventory = df[~(df['monster_n'] == 'Inventory')]
print(len(df_not_inventory))
df_not_inventory.value_counts('set')

343


set
Violent          110
Swift             49
Will              37
Vampire           24
Blade             19
Energy            17
Fatal             16
Nemesis            9
Rage               9
Fight              8
Despair            8
Guard              8
Revenge            8
Focus              8
Shield             6
Destroy            4
Enhance            1
Endure             1
Determination      1
dtype: int64

In [8]:
# Clean columns
df = df.drop(columns=['s1_t',	's1_v',	's1_data',
                      's2_t', 's2_v', 's2_data',
                      's3_t',	's3_v',	's3_data',
                      's4_t',	's4_v',	's4_data',
                      'DT_RowId',	'id',	'unique_id',	'monster',	'originID', 'originName', 'efficiency', 'max_efficiency', 'locked'])

cols_original_clean = df.columns

In [9]:
# convert inherent stats to eff values, 0-1.
# keep separate because these can't be increased like normal stats

inherent_label_to_sub_label = {'ACC': 'sub_acc', "RES": "sub_res", "ATK%": "sub_atkp", "ATK flat": "sub_atkf",  "DEF%": "sub_defp", "DEF flat": "sub_deff",  "HP%": "sub_hpp", "HP flat": "sub_hpf", "SPD": "sub_spd", "CRate": "sub_crate", "CDmg": "sub_cdmg"}

df['i_t_clean'] = df['i_t'].replace(inherent_label_to_sub_label)

substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

df['inh_norm'] = df['i_v'] / (df['i_t_clean'].replace(substats_max)) * (df['i_t_clean'].replace(weights)) 

In [10]:
# convert substats to eff values, 0-5
substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

for label in substats_max:
    df[label] = pd.to_numeric(df[label])

for label in substats_max:
    df[label+'_norm'] = df[label]/substats_max[label]*weights[label]


In [11]:
#specify the columns to sum
cols = [str(label+'_norm') for label in substats_max]
cols.append('inh_norm')
#find sum of columns specified 
df['tot_sum_norm'] = df[cols].sum(axis=1)

In [12]:
df['num_powerup_left'] = np.maximum(0, np.subtract(4, np.floor_divide(df['level'], 3)))
df['num_powerup_used'] = np.minimum(4, np.floor_divide(df['level'], 3))


df['num_powerup_incsub'] = df['quality'].map({'Unknown': 0, 'Rare': 2, 'Hero': 3, 'Legend': 4})

df['num_powerup_incsub_left'] = np.maximum(np.subtract(df['num_powerup_incsub'], df['num_powerup_used']) , 0)
df['num_powerup_newsub_left'] = np.subtract(df['num_powerup_left'], df['num_powerup_incsub_left'] )

In [13]:
# From increasing current substats:
# if spd is an increasable substat, then assume all rolls go there
# otherwise assume all rolls go to not-good stats
# TODO this roll could possibly only go to a bad stat (as per norm), not a good one. account for this

df['sub_inc_max_norm'] = (df['num_powerup_incsub_left']*2).where(~df['sub_spd'].isna(), df['num_powerup_incsub_left'])

# assume these are all going to bad stats.
# (even if speed is rollable, probability of going to spd is...low and not worth)
# TODO consider like max vs expected. this straddles the line somewhere
df['sub_new_max_norm'] = df['num_powerup_newsub_left']

In [14]:
# and now sum for the "max" roll eff
cols = ['tot_sum_norm', 'sub_inc_max_norm', 'sub_new_max_norm']

df['tot_max_norm'] = df[cols].sum(axis=1)

In [15]:
# worst cases:
# TODO due to the chance of flat rolls, this is lower.
# also like account for inc rolls can only be present stats; new rolls can only be one of each (not double-up into bad roll)
df['sub_inc_min_norm'] = df['num_powerup_incsub_left'] * 0.5 # (0.5 is worstcase eff from a roll)
df['sub_new_min_norm'] = df['num_powerup_newsub_left'] * 0.5
cols = ['tot_sum_norm', 'sub_inc_min_norm', 'sub_new_min_norm']
df['tot_min_norm'] = df[cols].sum(axis=1)

In [16]:
# for ease in using the outputted sheet
df['is_odd'] = df['slot'].apply(lambda x: (x%2!=0))

TIERS_TO_SETS = {'S': ['Violent'], 'A': ['Swift', 'Will'], 
'B': ['Blade', 'Fatal', 'Rage', 'Vampire', 'Focus', 'Shield', 'Nemesis', 'Despair', 'Revenge', 'Destroy'],
'C': ['Energy', 'Guard', 'Blade', 'Endure', 'Fight', 'Determination', 'Enhance', 'Accuracy', 'Tolerance']}

SETS_TO_TIERS = dict()
for tier in TIERS_TO_SETS:
    for set in TIERS_TO_SETS[tier]:
        SETS_TO_TIERS[set] = tier
print(SETS_TO_TIERS)

df['tier'] = df['set'].apply(lambda x: SETS_TO_TIERS[x])

# To scale to a normalized 0-to-1 scale
# df['tot_max_norm'] = (df['tot_max_norm']/9.round(3)

{'Violent': 'S', 'Swift': 'A', 'Will': 'A', 'Blade': 'C', 'Fatal': 'B', 'Rage': 'B', 'Vampire': 'B', 'Focus': 'B', 'Shield': 'B', 'Nemesis': 'B', 'Despair': 'B', 'Revenge': 'B', 'Destroy': 'B', 'Energy': 'C', 'Guard': 'C', 'Endure': 'C', 'Fight': 'C', 'Determination': 'C', 'Enhance': 'C', 'Accuracy': 'C', 'Tolerance': 'C'}


In [17]:
df['flag'] = False


GOAL_TOTAL = 3000 # start lower, but steadily raise to 3000.
TIER_RATIOS = {'S': 0.32, 'A': 0.25, 'B': 0.32, 'C': 0.11}
IS_ODD_RATIO = 1/3
CUTOFF_FACTOR = 0.8 # all held runes can _potentially_ be better than the n-th best rune. 
# where is this cutoff? where do you draw the line? by some scale:
# what is n divided by the number of total runes of class?
# Note that if scaling_factor=1, then you will hold more runes than the goal_total:
# you'll hold everything already better, and potentially better
# Fine-tune this as needed, something like 0.8-0.9 works well
# **Lower factor means holding less runes. But saves mana (since more selective w powerups)**
# SCALE SUCH THAT MANA STONES ARE MANAGEABLE 

GLOBAL_EFF_CUTOFF = 7 # any non-spd rune, less than this max eff are sold, regardless of the goal totals etc
# To prevent flooding inventory with runes that, even if they're _relatively_ good enough,
# would still be easily, quickly sold by obtaining better runes


for tier in ('S', 'A', 'B', 'C'):
    for is_odd in (True, False):
        print(tier, is_odd)

        goal_freq = int(GOAL_TOTAL * TIER_RATIOS[tier] * (IS_ODD_RATIO if is_odd else 1-IS_ODD_RATIO))
        cutoff_freq = int(goal_freq * CUTOFF_FACTOR)

        mask_subset = df['is_odd']
        if not is_odd:
            mask_subset = ~mask_subset
        mask_subset = mask_subset & (df['tier']==tier)

        # Figure out how many runes to sell: 
        # this is tricky, as we want to keep _all_ speed runes 
        # (due to quirks w efficiency, importance of speed, etc)
        if not is_odd:
            count_spd = sum(mask_subset & (df['m_t'] == 'SPD'))
            cutoff_freq -= count_spd
            goal_freq -= count_spd
            mask_subset = mask_subset & (df['m_t'] != 'SPD')

        df_subset = df[mask_subset]
        df_subset = df_subset.sort_values(by=['tot_min_norm'], ascending=False)

        print(len(df_subset), goal_freq, goal_freq-len(df_subset))
        # Calc cutoff, based on rune counts, and "n-th" best rune
        if len(df_subset) < cutoff_freq:
            # then want to keep everything. can't sell runes below the n'th-best if there aren't n total 
            # (effectively mark this by setting the "sell if below" threshold to 0)
            eff_cutoff = 0
        else:
            rune_cutoff = df_subset.iloc[cutoff_freq-1] # change to cutoff_freq
            eff_cutoff = rune_cutoff.tot_sum_norm

        # if eff cutoff less than global eff cutoff,
        # then enforce global instead
        if eff_cutoff < GLOBAL_EFF_CUTOFF:
            eff_cutoff = GLOBAL_EFF_CUTOFF
        print(eff_cutoff)

        # flag runes in the subset, below the cutoff
        # keep previous runes too
        df['flag'] = (mask_subset & (df['tot_max_norm'] < eff_cutoff)) | df['flag']


S True
97 320 223
7
S False
83 606 523
7
A True
119 250 131
7
A False
112 467 355
7
B True
267 320 53
7
B False
135 559 424
7
C True
142 110 -32
7
C False
79 159 80
7


## don't sell +15, or non-inventory

(sell only if not-15, and if in inventory)

In [18]:
df['sell'] = df['flag']
df['sell'] = df['sell'] & ((df['monster_n'] == 'Inventory'))
df['sell'] = df['sell'] & ((df['level'] != 15))

# Export: format data

In [19]:
cols_export = cols_original_clean
cols_export = cols_export.append(pd.Index(['tot_sum_norm', 'tot_min_norm', 'tot_max_norm', 'is_odd', 'tier', 'num_powerup_incsub_left', 'sell']))
df_export = df[cols_export]

In [20]:
df_export = df_export.sort_values(['set', 'grade', 'slot'], ascending=[True, False, True])
# df_export = df_export.sort_values('grade', ascending=False)
# df_export = df_export.sort_values('set')

In [21]:
# using datetime module
import datetime
  
# ct stores current time
ct = datetime.datetime.now()
ct = int(ct.timestamp())
df_export.to_csv(f'runes-data-out-{str(ct)}.csv', sep='\t')

In [22]:
df_export_to_sell = df_export[df_export['sell']]
print(df_export_to_sell)
df_export_to_sell.to_csv(f'runes-data-sell-{str(ct)}.csv', sep='\t')

      monster_n  ancient      set  slot  grade  level       m_t  m_v      i_t  \
1204  Inventory    False    Blade     4      6      0      CDmg   11      ACC   
1234  Inventory    False  Despair     1      6      0  ATK flat   22  HP flat   
1235  Inventory    False  Despair     3      6      0  DEF flat   22     DEF%   
1236  Inventory    False  Despair     5      6      0   HP flat  360      NaN   
1237  Inventory    False  Despair     5      6      0   HP flat  360     DEF%   
...         ...      ...      ...   ...    ...    ...       ...  ...      ...   
1005  Inventory    False     Will     2      5      0       HP%    8      NaN   
589   Inventory    False     Will     4      5     12      CDmg   48      NaN   
1020  Inventory    False     Will     4      5      0       HP%    8     CDmg   
620   Inventory     True     Will     6      5      9      DEF%   29      SPD   
1030  Inventory    False     Will     6      5      0      DEF%    8      HP%   

      i_v  ...  sub_spd  su

In [23]:
# TODO custom sort keys??
# TODO col of "reapp friendly"